In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf

from ngram import *
from baselines import *
from error_analysis import *

In [2]:
d_b, d_r =load_cf_data()
d_br = pd.concat([d_b, d_r])

In [3]:
max_features = 5000
architecture = [50, 20]

In [4]:
ngram_feature_pipeline = Pipeline([
    ('vect', CountVectorizer(ngram_range = (1,5), analyzer = 'char', max_features = max_features)),
    ('tfidf', TfidfTransformer(sublinear_tf=True,norm='l2')),
])

In [5]:
training_epochs = 75
batch_size = 500
display_step = 500

# BR on BR

In [6]:
X_train, y_train, X_test, y_test = tf_prep(d_br, ngram_feature_pipeline, 'aggression_score', average)

In [7]:
NN_REG(X_train,
        y_train,
        X_test,
        y_test,
        training_epochs = 100,
        batch_size = 500,
        display_step = 1000,
        architecture = architecture)

Batch: 0001 cost= 520.881347656

	Train RMSE:  21.099984638881573
	Test RMSE:  21.06386814223393

	Train R^2:  -3.62353914127
	Test R^2:  -3.6946814226


Batch: 1001 cost= 55.183299684

	Train RMSE:  1.4678748448951848
	Test RMSE:  1.5186005405766168

	Train R^2:  -0.10235701858
	Test R^2:  -0.12153357873


Batch: 2001 cost= 42.626442529

	Train RMSE:  0.8355617444179788
	Test RMSE:  0.866684027351025

	Train R^2:  -0.181365862203
	Test R^2:  -0.195034627593


Batch: 3001 cost= 30.923372904

	Train RMSE:  0.6232201374259531
	Test RMSE:  0.6524085671699373

	Train R^2:  0.0881790110061
	Test R^2:  0.0501215663512


Batch: 4001 cost= 21.065930358

	Train RMSE:  0.5100345284376745
	Test RMSE:  0.5400153059769109

	Train R^2:  0.363247978519
	Test R^2:  0.307620191937


Batch: 5001 cost= 12.589440621

	Train RMSE:  0.4453617270586492
	Test RMSE:  0.47592772604726385

	Train R^2:  0.521710667324
	Test R^2:  0.457856632617


Batch: 6001 cost= 6.938857513

	Train RMSE:  0.4133395125457447
	Te

In [9]:
sk_reg_model = Ridge(alpha = 1.0).fit(X_train, y_train)
_ = get_regression_metrics(sk_reg_model.predict(X_test), y_test)

R^2: 0.566
RMSE: 0.429
Mean absolute error: 0.317
Median absolute error: 0.239
Pearson: 0.835
Spearman: 0.744


# Random on Random

In [24]:
X_train, y_train, X_test, y_test = tf_prep(d_b, ngram_feature_pipeline, 'aggression_score', average)

(3999, 1)

In [25]:
NN_REG(X_train,
        y_train,
        X_test,
        y_test,
        training_epochs = 100,
        batch_size = 500,
        display_step = 1000,
        architecture = architecture)

Batch: 0001 cost= 1591.054687500

	Train RMSE:  40.05427712220673
	Test RMSE:  39.97872458692088

	Train R^2:  -12.6953108298
	Test R^2:  -12.5926494438


Batch: 1001 cost= 60.303840637

	Train RMSE:  1.7028794493942203
	Test RMSE:  1.8017602730189872

	Train R^2:  -0.0240289297425
	Test R^2:  -0.0348713970643


Batch: 2001 cost= 50.808229110

	Train RMSE:  0.8433082921767926
	Test RMSE:  1.0108507614701627

	Train R^2:  -0.0835647488017
	Test R^2:  -0.0776316936593


Batch: 3001 cost= 41.912141724

	Train RMSE:  0.5287524605212611
	Test RMSE:  0.7498003358278855

	Train R^2:  -0.219931160498
	Test R^2:  -0.127767962056


Optimization Finished!


In [30]:
sk_reg_model = Ridge(alpha = 1.0).fit(X_train, y_train.dot(classes))
_ = get_regression_metrics(sk_reg_model.predict(X_test), y_test)

R^2: -3.739
RMSE: 0.663
Mean absolute error: 0.544
Median absolute error: 0.480
Pearson: -0.004
Spearman: -0.020
